In [1]:
import torch

In [2]:
import tensorflow as tf

import pandas as pd
 
df = pd.DataFrame({'Department1':[78,16,89],
                   'Department2': ['Science','Maths','Biology']})
 
df
new_df_val = tf.data.Dataset.from_tensor_slices(dict(df))
new_df_val 
for i in new_df_val .take(3):
    print(i)

{'Department1': <tf.Tensor: shape=(), dtype=int64, numpy=78>, 'Department2': <tf.Tensor: shape=(), dtype=string, numpy=b'Science'>}
{'Department1': <tf.Tensor: shape=(), dtype=int64, numpy=16>, 'Department2': <tf.Tensor: shape=(), dtype=string, numpy=b'Maths'>}
{'Department1': <tf.Tensor: shape=(), dtype=int64, numpy=89>, 'Department2': <tf.Tensor: shape=(), dtype=string, numpy=b'Biology'>}


In [3]:
df = pd.DataFrame({'Department1':[178,965,156],
                   'Department2': ['Chemistry','Maths','Biology']})
 
df
new_df_val = tf.data.Dataset.from_tensor_slices(dict(df))
new_df_val 
for i in new_df_val .take(3):
    print(i)

{'Department1': <tf.Tensor: shape=(), dtype=int64, numpy=178>, 'Department2': <tf.Tensor: shape=(), dtype=string, numpy=b'Chemistry'>}
{'Department1': <tf.Tensor: shape=(), dtype=int64, numpy=965>, 'Department2': <tf.Tensor: shape=(), dtype=string, numpy=b'Maths'>}
{'Department1': <tf.Tensor: shape=(), dtype=int64, numpy=156>, 'Department2': <tf.Tensor: shape=(), dtype=string, numpy=b'Biology'>}


In [4]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
from urllib import request as rq
import pandas as pd
import os
import pyarrow as pa # this is needed for the parquet file
import numpy as np
import ipywidgets
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, VBox
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns



In [8]:
# Function to load the Dublin bus gz files 
def load_Files(direc, files, comtype):
    columns = ['Timestamp', 'LineID', 'Direction', 'JourneyPatternID', 'TimeFrame', 'VehicleJourneyID', 'Operator', 'Congestion', 'LonWGS84', 'LatWGS84', 'Delay', 'BlockID', 'VehicleID', 'StopID', 'AtStop']
    for f in files:
        print(f)
        yield pd.read_csv(direc + f, compression=comtype, delimiter=',', header=0, names=columns, parse_dates=True, low_memory=True)

def crackit_open(busFile):
    import zipfile as zip
    # Zip creates its own folders - no need to check for folder existence
    with zip.ZipFile(busFile,  mode='r') as arc: 
        arc.extractall('./Data/Bus/Gz/')  
    files = os.listdir('./Data/Bus/Gz/')
    DBfiles = [f for f in files if f.endswith('.gz')]
    df = pd.concat(load_Files('./Data/Bus/Gz/', DBfiles, 'gzip'), copy = False)
    return df

def shapiro_test(x):
    p_val = stats.shapiro(x)[1]
    status = 'passed'
    color = 'blue'
    if p_val < 0.05:
        status = 'failed'
        color = 'red'
    return status, color, p_val

def custom_scatterplot(df1, col1=''):
    df1 = df1[df1["LineID"]==col1]
    f = plt.figure()
    f, ax = plt.subplots(figsize=(11.5, 11.5))
    ax = f.add_subplot(projection='3d')
    ax.scatter(df1['LonWGS84'], df1['LatWGS84'], df1['Hour'], alpha=0.6, color=df1['Colour'])
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_zlabel('Hour')
    dcol = str(col1)
    plt.savefig('./Images/Img_' + dcol + '_Longitude_Latitude_Hour.svg')
    df1.to_parquet('./Data/Bus/LineID_' + dcol + '.parquet')
    
    
def custom_barplot(df1, col1=''):
    if len(df1[col1]) > 5000: # added this to the function because of warnings about the size of data being used with shapiro test
            sampleSize = 5000
    else:
        sampleSize = len(df1[col1])
    df1 = df1.sample(sampleSize) #shapiro test is unreliable over 5000 https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test and performance reasons
    f, ax = plt.subplots(2,2, figsize=(11.5, 11.5))
    ax = ax.reshape(-1)
    df1[col1].plot(ax=ax[0], kind='hist')
    ax[0].set_title('Histogram of {}'.format(col1))
    df1[col1].plot(ax=ax[1], kind='kde')
    ax[1].set_title('Density Plot of {}'.format(col1))
    ax3 = plt.subplot(223)
    stats.probplot(df[col1], plot=plt)
    ax[2].set_title('QQ Plot of {}'.format(col1))
    df1[col1].plot(ax=ax[3], kind='box')
    ax[3].set_title('Box Plot of {}'.format(col1))
    status, color, p_val = shapiro_test(df1[col1]) 
    f.suptitle('Normality test for {} {} (p_value = {})'.format(col1, status, p_val), color=color, fontsize=12)

def num_missing(x):
    return len(x.index)-x.count()

def num_unique(x):
    return len(np.unique(x))

def load_csv_Files(direc, files):

    for f in files:
        # need to get number of rows to skip 
        temp=pd.read_csv(direc + f,sep='^',header=None,prefix='X')
        temp2=temp.X0.str.split(',',expand=True)
        del temp['X0']
        temp=pd.concat([temp,temp2],axis=1)
        cols = list(range(0,temp.shape[1]))

        print(f)
        yield pd.read_csv(direc + f,  delimiter=',', header=0,  parse_dates=True, low_memory=True, skiprows=14, usecols=cols, na_values='NAN')

In [9]:
if os.path.exists('./Data'):
    print('Data folder exists')

if os.path.exists('./Data/Bus'):
    print('Bus Data folder exists')
else:
    os.makedirs('./Data/Bus')

if os.path.exists('./Data/MetEirrean/'):
    print('Weather data folder exists')
else:
    os.makedirs('./Data/MetEirrean/')

if os.path.exists('./Zips/MetEirrean'):
    print('Weather folder exisits')
else:
    os.makedirs('./Zips/MetEirrean/')

if os.path.exists('./Images/'):
    print('Images folder exists')
else:
    os.makedirs('./Images/')



Data folder exists


In [10]:
# Check parquet file existence before downloading - iof starting from afresh this takes a long time
if os.path.exists('./Data/CleanedBusData.parquet'):
    print("Parquest file exists, means the Bus data has been downloaded already ")
    df = pd.read_parquet('./Data/CleanedBusData.parquet')
    if os.path.exists('./Data/WeatherandBusData.parquet'):
        print('Weather and bus data combined exists')
        mdf = pd.read_parquet('./Data/WeatherandBusData.parquet')
elif os.path.exists('./Zips/Bus/DublinBusdata.zip'):
    print("Zip file exists, we have already downloaded the Dublin Bus Zip data, crack it open")
    df = crackit_open('./Zips/Bus/DublinBusdata.zip')
else:
    os.makedirs('./Zips/Bus/', exist_ok = True)
    url = "https://opendata.dublincity.ie/TrafficOpenData/sir010113-310113.zip"
    busFile = rq.urlretrieve(url, './Zips/Bus/DublinBusdata.zip' )  
    df = crackit_open('./Zips/Bus/DublinBusdata.zip')

df = df.sample(20000000)
### Read the Bus data in to a Pandas dataframe - done

Parquest file exists, means the Bus data has been downloaded already 


In [11]:
if 'Direction' in df.columns:
    df = df.drop(['Direction'], axis='columns')
if 'TimeFrame' in df.columns:
    df = df.drop(['TimeFrame'], axis='columns')
""" if {'VehicleJourneyID', 'JourneyPatternID'}.issubset(df.columns):  
    df['VehicleJourneyID'] = df['JourneyPatternID'] + '_' + df['VehicleJourneyID'].astype('str')
    df = df.drop(['JourneyPatternID','VehicleJourneyID'], axis='columns') """

df = df.dropna() 
df = df.drop_duplicates()

temp_df = df.describe().T
missing_df = pd.DataFrame(df.apply(num_missing, axis=0)) 
missing_df.columns = ['missing']  # type: ignore

display(temp_df)

display(missing_df)



,count,mean,std,min,25%,50%,75%,max
Timestamp,19998897.0,1.358318e+15,7.395001e+11,1.356998e+15,1.357679e+15,1.358327e+15,1.358955e+15,1.359633e+15
LineID,19998897.0,7.796954e+01,1.146472e+02,1.000000e+00,2.500000e+01,4.000000e+01,8.300000e+01,7.470000e+02
VehicleJourneyID,19998897.0,9.415629e+03,6.159182e+04,1.000000e+00,2.535000e+03,4.718000e+03,6.769000e+03,9.998560e+05
Congestion,19998897.0,1.164869e-02,1.072987e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
LonWGS84,19998897.0,-6.272790e+00,8.388212e-02,-6.617434e+00,-6.308630e+00,-6.261595e+00,-6.233166e+00,-6.052917e+00
LatWGS84,19998897.0,5.334513e+01,5.486668e-02,5.306802e+01,5.332004e+01,5.334645e+01,5.337532e+01,5.360870e+01
Delay,19998897.0,-2.814680e+01,4.734957e+02,-1.504500e+04,-2.090000e+02,0.000000e+00,1.040000e+02,1.161220e+05
BlockID,19998897.0,1.092300e+05,1.921073e+05,3.900000e+02,1.602000e+04,4.020500e+04,8.400400e+04,8.350020e+05
VehicleID,19998897.0,3.542959e+04,3.281788e+03,2.804700e+04,3.330800e+04,3.352400e+04,3.802500e+04,4.307800e+04
AtStop,19998897.0,2.334375e-01,4.230183e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


,missing
Timestamp,0
LineID,0
VehicleJourneyID,0
Operator,0
Congestion,0
LonWGS84,0
LatWGS84,0
Delay,0
BlockID,0
VehicleID,0


In [12]:
print ('The data has {} Rows and {} columns'.format(df.shape[0],df.shape[1]))
print("The types of columns are:")
display(df.dtypes)

The data has 19998897 Rows and 16 columns
The types of columns are:


Timestamp                    int64
LineID                     float64
VehicleJourneyID             int64
Operator                    object
Congestion                   int64
LonWGS84                   float64
LatWGS84                   float64
Delay                        int64
BlockID                      int64
VehicleID                    int64
AtStop                       int64
datetime            datetime64[ns]
Time                        object
Day                          int64
Hour                         int64
Minute                       int64
dtype: object

In [16]:
df = df.drop(['datetime'], axis='columns')

In [23]:
colNames = list(df.columns)

In [21]:
display(df.dtypes)

Timestamp             int64
LineID              float64
VehicleJourneyID      int64
Operator             object
Congestion            int64
LonWGS84            float64
LatWGS84            float64
Delay                 int64
BlockID               int64
VehicleID             int64
AtStop                int64
Time                 object
Day                   int64
Hour                  int64
Minute                int64
dtype: object

In [33]:
df = df.astype({'Timestamp': 'float64', 'VehicleJourneyID': 'float64',  'Congestion': 'float64', 'Delay': 'float64', 'BlockID': 'float64', 'VehicleID': 'float64',  'AtStop': 'float64', 'Day': 'float64', 'Hour':'float64', 'Minute': 'float64' })

In [34]:
df.dtypes

Timestamp           float64
LineID              float64
VehicleJourneyID    float64
Operator             object
Congestion          float64
LonWGS84            float64
LatWGS84            float64
Delay               float64
BlockID             float64
VehicleID           float64
AtStop              float64
Time                 object
Day                 float64
Hour                float64
Minute              float64
dtype: object

In [35]:
new_df_val = tf.data.Dataset.from_tensor_slices(df)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).